#  Lunar Lander with Least Squares Policy Iteration (LSPI) :

Author: Nirav Lad

This notebook implements [Lunar Lander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) as available through [Farama Foundation Gymnasium](https://gymnasium.farama.org/).  The algorithm utilized is [LSPI](https://www.jmlr.org/papers/volume4/lagoudakis03a/lagoudakis03a.pdf) with a some differences as noted (e.g. weight updates) from the paper by [Lagoudakis](https://scholar.google.com/citations?user=VqHiIg8AAAAJ) and [Parr](https://scholar.google.com/citations?user=b-GJ3QIAAAAJ).

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import itertools

import gymnasium as gym

from typing import List, Tuple, Union, Callable
from numpy.typing import NDArray
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from gymnasium.wrappers import RecordVideo
from moviepy import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips
from IPython.display import Video
import os
import shutil

## Define parameters

### Enviroment variables

In [2]:
ENV_NAME = 'LunarLander-v3'

In [3]:
env = gym.make(ENV_NAME)
cardS = len( env.observation_space.low )
A = np.arange( env.action_space.n )
s_low = env.observation_space.low
s_range = env.observation_space.high - env.observation_space.low
env.close()

num_best_policies = 10

### Hyperparameters

In [4]:
Δ = 1e-6
γ = 0.9
lr = (0.5, 0.5)

## Functions

### Mean and halfwidth calculation

In [5]:
def calculate_mean_hw( d: Union[List[np.float32] | NDArray[np.float32]], ci:int = 0.95 ) -> Tuple[np.float32, np.float32]:
    d = np.array( d )
    n = np.size( d )
    std_error = np.std( d, ddof=1 ) / np.sqrt(n)
    t_critical = stats.t.ppf( 1 - ( 1 - ci ) / 2, n - 1 )
    mean = np.mean( d ) 
    halfwidth = t_critical * std_error
    return( mean, halfwidth )

### Basis functions

Each row of $\Phi$ contains the value of all basis functions for a certain pair $(s,a)$ and each
column of $\Phi$ contains the value of a certain basis function for all pairs $(s,a)$. Using the indicator function $I$ then we have $\Phi(s,a) = \begin{pmatrix} \phi(s,a)\times I(a=1), \cdots, \phi(s,a)\times I(a=k)\end{pmatrix}^{\intercal}$

In [6]:
def Φ( state:NDArray[np.float32], action:int, φ:Callable[[NDArray[np.float32], int], int] ) -> NDArray[np.float32]:
    return( np.hstack([ φ(state, action ) * ( a == action ) for a in A ]) )

### Poilicy function ($\pi$)

$\pi(s) =\arg \max_{a \in \mathcal{A}} \Phi(s,a)^{\intercal} w$

In [7]:
def π( state:NDArray[np.float32], w:NDArray[np.float32], φ:Callable[[NDArray[np.float32],int],int] ) -> int:
    return( np.argmax( [ np.dot( Φ( state, a, φ ), w ) for a in A ]) )

### TD Learning rate

The learning rate at episode $n$ with initial learnign rate $\alpha_a$ and learning rate decay $\alpha_b$ is defined as $\alpha_n =  a_{a}\left( \frac{1}{(1 + n)^{\alpha_{b}}} \right)$

In [8]:
def α( n:int, lr:Tuple[int,int] ) -> float:
    α_init, α_decay = lr
    return( α_init / (n + 1)**α_decay )

### Generate random samples

In [9]:
def get_samples( w:NDArray[np.float32], 
                 φ:Callable[[NDArray[np.float32], int],int],
                 num_samples:int = int(2**7), 
                 max_samples:int = int(2**11), 
                 seed:int = 42, 
                 enable_wind:bool = False ) -> List[Tuple]:

    env = gym.make( ENV_NAME, enable_wind = enable_wind )

    np.random.seed( int( seed * 1e3 ) )

    samples = []

    while ( len(samples) <  max_samples ):

        terminated = False
        truncated = False

        s = env.reset( seed = int( seed * 1e3 ) )[0]

        while not( terminated or truncated ):
            a = π( s, w, φ )  
            sP, r, terminated, truncated, _ = env.step( a )
            samples.append( (s, a, r, sP, terminated) )
            s = sP
            if ( len(samples) == max_samples ):
                break

    env.close()

    indicies = np.random.choice( len(samples), size=num_samples, replace=False )
    random_samples = [ samples[i] for i in indicies ]

    return( random_samples )

### Evaluate policy

In [10]:
def evaluate_policy( w:NDArray[np.float32], 
                     φ:Callable[[NDArray[np.float32], int],int],
                     num_evaluations:int = 10, 
                     seed:int = 42, 
                     enable_wind:bool = False ) -> Tuple[float, float]: 

    env = gym.make( ENV_NAME, enable_wind = enable_wind )

    G = []

    for n in np.arange( num_evaluations ):
        terminated = False
        truncated = False

        Gm = 0
        s = env.reset( seed = int( seed * 1e2 + n * 3e4 ) )[0]

        while not( terminated or truncated ):
            a = π( s, w, φ )
            sP, r, terminated, truncated, _ = env.step( a )
            s = sP
            Gm += r

        G.append( Gm )

    μ, δ = calculate_mean_hw( G )

    return ( μ, δ )

### **LSTDQ**-opt

Implementation of <b>LSDQ</b>-opt that utilizes the [Sherman-Morris formula](https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula) to compute the "rank-1 update" for matrix inversion. This allows computation in $O(n^2)$ as opposed to $O(n^3)$.

<b>Inputs</b>:
- $D$: Source of samples $(s,a,r,s')$
- $k$: Number of basis functions
- $\Phi$: Basis functions
- $\gamma$: Discount factor
- $\pi$: Policy whose value function is sought

<b>algorithm</b>:<br>
1: $\mathbf{B} \gets \frac{1}{\Delta} \mathbf{I} \;\;$ is a $(k \times k)$ matrix<br>
2: $b \gets \mathbf{0} \;\;$ is a $(k \times 1)$ matrix<br>
3: <b>for each</b> $(s,a,r,s') \in D$<br>
4: &nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{B} \gets \mathbf{B} - \frac{ \mathbf{B}\Phi(s,a)\left( \Phi(s,a) - \gamma \Phi(s', \pi(s')) \right)^{\intercal} \mathbf{B} }{ 1 + \left( \Phi(s,a) - \gamma \Phi(s',\pi(s')) \right)^5: {\intercal} \mathbf{B} \Phi(s,a)}$<br>
5: &nbsp;&nbsp;&nbsp;&nbsp;$b \gets b + \Phi(s,a)r$<br>
6: $\tilde{w}^{\pi} \gets \mathbf{B}b$ results in a $(k \times 1)$ matrix<br>
7: <b>return<b> $\tilde{w}^{\pi}$

In [11]:
def LSTDQ_opt( D:List[Tuple], 
               k:int,
               w:NDArray[np.float32], 
               φ:Callable[[NDArray[np.float32], int], int], 
               n:int, 
               lr:Tuple[int,int] ) -> NDArray[np.float32]:

    B = np.eye( k ) * Δ
    b = np.zeros( k )

    for ( s, a, r, sP, terminated ) in D:

        Φ_sa = Φ( s, a, φ ).flatten()
        Φ_sPaP = Φ( sP, π( sP, w, φ ), φ ).flatten() * ( 1 - terminated ) 
        
        u = B @ Φ_sa
        v = ( Φ_sa - γ * Φ_sPaP ).T @ B
        
        B = B - np.outer( u, v ) / ( 1 + v @ Φ_sa )
        b = b +  r * Φ_sa
    
    wP = B @ b.T
    wP = np.asarray( wP ).ravel()

    w = ( 1 - α(n,lr) ) * w + ( α(n,lr) )* wP

    return( w )

### **LSPI**

The stopping criteria for this implementation is stopping after $M$ episodes. Additionally, the function returns all training policies for later evaluation rather than the superlative policy $w^*$.

<b>Inputs</b>:
- $D$: Source of samples $(s,a,r,s')$
- $k$: Number of basis functions
- $\Phi$: Basis functions
- $\gamma$: Discount factor
- $\epsilon$: Stopping criterion
- $\pi_0$: Initial policy, given as $w_0$ (default $w_0 = \mathbf{0}$)

<b>algorithm</b>:<br>
1: $\pi' \gets \pi_0$<br>
2: <b>repeat</b><br>
3: &nbsp;&nbsp;&nbsp;&nbsp;$\pi \gets \pi'$<br>
4: &nbsp;&nbsp;&nbsp;&nbsp;$\pi' \gets$ <b>LSTDQ</b> $(D,k,\Phi, \gamma, \pi)$<br>
5: <b>until</b>$(\pi \approx \pi')$ <br>
6: <b>return<b> $\pi$

In [12]:
def LSPI( w:NDArray[np.float32], 
          k:int, 
          φ:Callable[[NDArray[np.float32], int], int], 
          lr:Tuple[int,int], 
          M:int = 500, 
          seed:int = 0, 
          enable_wind:bool=False,  
          num_samples = int(2**4), 
          max_samples=int(2**11), ) -> pd.DataFrame:

    d = []

    for m in tqdm( np.arange(M) ):

        D = get_samples( w, φ, seed=( m + seed ), num_samples = num_samples, max_samples=max_samples, enable_wind=enable_wind )

        w = LSTDQ_opt( D, k, w, φ, m, lr )
        
        μ, δ = evaluate_policy( w, φ, seed = m, enable_wind=enable_wind )

        score = μ - δ

        tqdm.write(f"Episode{m:3d}:\t{score:.2f}\t({μ:3.2f} ± {δ:3.2f})")
        d.append({'episode':m, 'score': score, 'mean': μ, 'halfwidth': δ, 'w': w.copy()})

    policies = pd.DataFrame( d )

    return( policies )

### Record policy video

In [13]:
def record_policy( w:NDArray[np.float32], 
                   φ:Callable[[NDArray[np.float32], int],int],
                   title:str, 
                   M:int = 10, 
                   seed:int = 1982 ) -> str:

    import warnings
    warnings.filterwarnings("ignore", message=".*Overwriting existing videos.*")
    os.makedirs("videos", exist_ok=True)
    env = gym.make(ENV_NAME, render_mode="rgb_array")
    env = RecordVideo(env, video_folder="videos", episode_trigger=lambda ep: True)

    G = []

    for m in np.arange(M):

        Gm = 0
        s = env.reset(seed= int( seed + m * 1e4 ) )[0]
        terminated = False
        truncated = False
        while not( terminated or truncated ):
            a = π( s, w, φ )
            sP, r, terminated, truncated, _ = env.step( a )
            s = sP
            Gm += r
        
        G.append( Gm )

    μ, δ = calculate_mean_hw( G )
    score = μ - δ

    env.close()

    video_files = sorted([f for f in os.listdir("videos") if f.endswith(".mp4")])
    annotated_clips = []
    for i, vf in enumerate(video_files):
        clip = VideoFileClip(f"videos/{vf}", )
        
        txt = TextClip( text=f"{title}\nEP {i+1:2d} Reward: {G[i]:0.1f}\nScore: {score:0.1f} ({μ:0.1f} ± {δ:0.1f})", 
                        font_size=14, 
                        color='white', 
                        font='DejaVuSansMono', 
                        method='caption',
                        vertical_align='left', horizontal_align='top',
                        size = (clip.w, 60) )
        txt = txt.with_duration(clip.duration)
        
        annotated_clip = CompositeVideoClip([clip, txt])
        annotated_clips.append(annotated_clip)

    final_clip = concatenate_videoclips(annotated_clips )

    fname = f"best_policy_({score:0.1f}).mp4"
    final_clip.write_videofile(fname)

    shutil.rmtree("videos")

    return( fname )

## Run **LSPI**

### Basis Function

In [14]:
def linear( state:NDArray[np.float32], action:int ) -> NDArray[np.float32]:

    px, py, vx, vy, θ, ω, lf, rf = state

    interactions = np.array([px*py, px*vx, px*θ, px*ω, px*lf, px*rf,
                             py*vx, py*vy, py*θ, py*ω, py*lf, py*rf,
                             vx*vy, vx*θ, vx*ω, vx*lf, vx*rf,
                             vy*θ, vy*ω, vy*lf, vy*rf,
                             θ*ω, θ*lf, θ*rf,
                             θ*lf, θ*rf,
                             lf*rf])
    
    return( np.hstack( ( state, interactions ) ) )

In [15]:
bf = linear

In [ ]:
k = len( Φ(np.zeros(cardS), A[0], bf) )
w = np.zeros( k )
policies = LSPI( w, k, bf, lr, M=100, enable_wind=False, num_samples=100, max_samples=1000 )

### Plot learning curve

In [17]:
plt.figure(figsize=(12, 6))  
sns.set_theme(style="whitegrid") 
sns.lineplot(data=policies, x='episode', y='mean', label="$\mu$")
plt.fill_between( policies['episode'],
                  policies['mean'] - policies['halfwidth'],
                  policies['mean'] + policies['halfwidth'],
                  alpha=0.3,
                  label='95% CI')

policies.sort_values(by='score', ascending=False, inplace=True)
p = policies.iloc[0]

plt.title(f"LSPI Learning Curve\n" +\
          f"Basis fuction:{bf.__name__},$\\Delta$={Δ:.2e}, $\\gamma$={γ:0.3f}, lr=({lr[0]:0.3f},{lr[1]:0.3f})\n" + \
          f"Best Score: {p['score']:.1f} ({p['mean']:.1f}$\\pm${p['halfwidth']:.1f}) @ Episode {p['episode']}" )
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.xlim(0, len(policies))
plt.ylim(-600, 300)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(10))
plt.legend(loc="lower right")
plt.tight_layout()

plt.savefig(f"learning_curve.pdf")

plt.show()

### Determine best policy ($w^*$)

In [18]:
best_score = -np.inf

print(f"EVALUATING TOP {num_best_policies} POLICES")
print("\t\tScore\t(μ ± δ)")
for i, p in policies.head(num_best_policies).iterrows():
    μ, δ = evaluate_policy(p['w'], bf, num_evaluations=30, seed=1776)
    score = μ - δ
    print(f"Episode {i:3d}\t{score:.1f}\t({μ:.1f} ± {δ:.1f})")
    if ( best_score < score ):
        best_score = score
        w_star = p['w']
        m_star = p['episode']

print(f"SUPERLATIVE POLICY:\tEPISODE {m_star}")

EVALUATING TOP 10 POLICES
		Score	(μ ± δ)
Episode  89	228.4	(238.9 ± 10.5)
Episode  90	199.7	(216.3 ± 16.6)
Episode  83	187.2	(210.9 ± 23.7)
Episode  33	104.8	(152.2 ± 47.3)
Episode  84	186.0	(200.0 ± 13.9)
Episode  82	200.7	(224.9 ± 24.2)
Episode  52	114.8	(138.1 ± 23.3)
Episode  34	32.1	(105.5 ± 73.4)
Episode  36	65.2	(111.1 ± 45.8)
Episode  94	155.8	(202.3 ± 46.5)
SUPERLATIVE POLICY:	EPISODE 89


### The superlative policy $w^*$ weights

In [19]:
np.set_printoptions(precision=2)
print(f"w*:")
print( w_star.reshape(-1,4) )

w*:
[[ 2.24e-06 -1.66e-05  2.96e-06  3.51e-05]
 [ 1.28e-05  1.56e-05 -2.60e-06 -4.03e-06]
 [-3.88e-07 -2.69e-06  2.01e-06 -9.24e-08]
 [-9.45e-08  2.31e-06  1.19e-06  9.62e-06]
 [ 3.57e-07  5.27e-07 -3.85e-07  3.15e-07]
 [-4.03e-06  5.37e-06  2.70e-06 -6.24e-07]
 [ 3.17e-06 -2.60e-05 -2.90e-05  5.77e-06]
 [ 9.34e-06 -4.37e-05  1.07e-06 -3.09e-06]
 [ 1.07e-06 -3.09e-06  9.70e-07  6.72e-06]
 [-1.06e-05  1.35e-05  3.61e-05  1.89e-06]
 [ 1.46e-05 -1.38e-05 -5.40e-06  1.68e-06]
 [-4.57e-06 -2.83e-06 -8.05e-06  3.70e-06]
 [ 2.81e-07  8.91e-06  1.70e-05 -5.57e-06]
 [-7.15e-06  9.90e-08  1.90e-07 -1.83e-05]
 [ 5.30e-06 -3.77e-06  4.07e-06 -1.53e-06]
 [-1.42e-06 -1.06e-05  1.47e-05  2.13e-06]
 [-4.41e-05  8.74e-06  2.42e-08  8.74e-06]
 [ 2.42e-08 -5.65e-06  9.46e-06 -2.31e-05]
 [ 1.17e-05  1.87e-05  2.58e-06  4.51e-06]
 [-1.03e-05 -7.33e-06  3.61e-06 -7.63e-06]
 [ 3.13e-06 -1.02e-06  3.81e-06  2.99e-06]
 [ 8.36e-06 -5.05e-06 -2.18e-06 -1.61e-06]
 [ 7.88e-07  8.11e-07 -9.06e-06  3.69e-05]
 [ 1.49

### View video recording of the best policy

In [ ]:
title = f"LSPI Δ={Δ:.2e}, γ={γ:0.3f}, lr=({lr[0]:0.3f},{lr[1]:0.3f})"
video_filename = record_policy( w_star, bf, title )
print(f"Done! Video saved to {video_filename}")

In [21]:
Video(video_filename, embed=True)